# Train car detector 
https://www.cs.toronto.edu/~kriz/cifar.html
<img src="../doc/cifar_dataset.png" alt="Cifar Dataset" width="300"/>

In [ ]:
# imports
import albumentations as A
import cv2

import keras
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Force CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
# Get GPU details
print(tf.test.gpu_device_name())
!nvidia-smi

In [ ]:
# Get data
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Make sample small for testing @todo remove
if False:
    x_train = x_train[:500]
    y_train = y_train[:500]
    x_test = x_test[:100]
    y_test = y_test[:100]

In [ ]:
# Scale image
x_train = x_train / 255.

In [ ]:
# Visualize data
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(x_train[3])
plt.show()

In [ ]:
# Create CNN and add layers
model = Sequential()

# Stack 1
model.add( Conv2D(32,
                 kernel_size=(3,3),
                 input_shape=(32, 32, 3), # Only needed for first layer
                 activation="relu",
                 padding="same"))
model.add( Conv2D(32,
                 kernel_size=(3,3),
                 activation="relu") )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout( 0.25 ) )

# Stack 2
model.add( Conv2D(64,
                 kernel_size=(3,3),
                 activation="relu") )
model.add( Conv2D(64,
                 kernel_size=(3,3),
                 activation="relu") )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout( 0.25 ) )

# Final Stack
model.add( Flatten() )

model.add( Dense(256, activation="relu") )
model.add( Dense(128, activation="relu") )

model.add( Dense(1, activation="sigmoid") )

In [ ]:
# Add optimizer to CNN
model.compile( optimizer="rmsprop",
               loss="binary_crossentropy",
               metrics=["accuracy"])

In [ ]:
# Train CNN (for cars)
y_train_car = y_train == 1 # 1: car
y_test_car = y_test == 1

model.fit(x_train, y_train_car, batch_size=128, epochs=10, shuffle=True, workers = 10)

In [ ]:
# Compare model with test data
print(model.summary())
print(model.evaluate(x_train, y_train_car))
print(model.evaluate(x_test, y_test_car))

In [ ]:
# Check GPU 
# https://www.tensorflow.org/guide/gpu
# https://medium.com/@kegui/how-do-i-know-i-am-running-keras-model-on-gpu-a9cdcc24f986
# 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Optimize CNN by creating additional training data

In [ ]:
# Train CNN (for cars)
y_train_car = y_train == 1 # 1: car
y_test_car = y_test == 1

In [ ]:
# Create additional data
# zoom, shift, etc. images
gen = ImageDataGenerator(width_shift_range=3,
                         height_shift_range=3,
                         zoom_range=0.1,
                         horizontal_flip=True) #,
                         #featurewise_center=True, # Caution: Apply this op to new images
                         #featurewise_std_normalization=True) # ... same here

gen.flow(x_train, y_train)

# Visualize data by iterating through it
# for batch in gen.flow(x_train, y_train, shuffle=False):
#     first_image = batch[0][0]
#     first_image = batch[0][1]
#     plt.imshow(first_image)
#     break

# If you change featurewise_center / *_std_normalizatoin you need to save them
# As you need to apply this operation on future images (to be detected)
# gen.__dict__

In [ ]:
# Add SPECIFIC optimizer to CNN
model2 = keras.models.clone_model(model)
model2.compile( optimizer="rmsprop",
               loss="binary_crossentropy",
               metrics=["accuracy"])

In [ ]:
# compare before "model.fit(x_train, y_train_car)"
model2.fit_generator(gen.flow(x_train, y_train_car, batch_size=128, shuffle=True), 
                    epochs=10, workers=10)

In [ ]:
# Save model
model.save('/tmp/cardetector.h5')
model2.save('/tmp/cardetector2.h5')

In [ ]:
# abumeration
